In [1]:
import pandas as pd
dataframe = pd.read_csv (r'/home/yupaporn/CCA_Project/Dataset_CCA_case_Trainset5257.csv')
dataframe

,Unnamed: 0,Case,Abs Position,Sub Position,Class,Path Full,Path Crop,Views,fold,15AB,...,height,Rleft,Rtop,Rwidth,Rheight,Sub_class_New,tag_AjNit,tag_AjWan,Test150,Spilt
0,0,AB01_15,P3,P31,Abnormal,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-B,3,MildFattyLiver,...,584.0,0.036832,0.107041,0.712707,0.649402,AB01,NaN,NaN,False,Train
1,1,AB01_15,P1,P1,Abnormal,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-A,3,MildFattyLiver,...,614.0,0.143898,0.107041,0.595628,0.683267,AB01,NaN,NaN,False,Train
2,2,AB01_15,P2,P2,Abnormal,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-A,3,MildFattyLiver,...,451.0,0.245009,0.113017,0.497278,0.501992,AB01,NaN,NaN,False,Train
3,3,AB01_15,P4,P42,Abnormal,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-B,3,MildFattyLiver,...,611.0,0.155235,0.122977,0.718412,0.679283,AB01,NaN,NaN,False,Train
4,4,AB01_15,P4,P41,Abnormal,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-B,3,MildFattyLiver,...,519.0,0.152057,0.162817,0.608229,0.577689,AB01,NaN,NaN,False,Train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5252,5252,344,P3,P32,Normal,/media/tohn/HDD/VISION_dataset/USAI/US images ...,/media/tohn/HDD/VISION_dataset/USAI/US images ...,FP-B,4,Normal,...,NaN,NaN,NaN,NaN,NaN,Normal,NaN,NaN,False,Train
5253,5253,344,P4,P42,Normal,/media/tohn/HDD/VISION_dataset/USAI/US images ...,/media/tohn/HDD/VISION_dataset/USAI/US images ...,FP-B,4,Normal,...,NaN,NaN,NaN,NaN,NaN,Normal,NaN,NaN,False,Train
5254,5254,344,P5,P52,Normal,/media/tohn/HDD/VISION_dataset/USAI/US images ...,/media/tohn/HDD/VISION_dataset/USAI/US images ...,FP-C,4,Normal,...,NaN,NaN,NaN,NaN,NaN,Normal,NaN,NaN,False,Train
5255,5255,344,P6,P61,Normal,/media/tohn/HDD/VISION_dataset/USAI/US images ...,/media/tohn/HDD/VISION_dataset/USAI/US images ...,FP-C,4,Normal,...,NaN,NaN,NaN,NaN,NaN,Normal,NaN,NaN,False,Train


# แบ่งข้อมูล

- ขนาด 2 ใน 3 ของข้อมูลเดิม
- Bagging ที่ควรกล่าวถึงคือจำนวนผู้เรียนควรมีจำนวนมากโดยปกติจะอยู่ระหว่าง 100 ถึง 500 ตัวแยกประเภทในเวลาเดียวกันแต่ละคนต้องได้รับความแม่นยำมากกว่า 50%



In [2]:
import os
import numpy as np
import pandas as pd
from scipy.special import softmax

In [7]:
def bayes_softmax(tb):
    df_ = tb
    conf_mat = pd.crosstab(df_['Views'],df_['Sub_class_New'],rownames=['5-FP Class'],colnames=['15AB Class'])

    # P(โรค|มุม) = P(มุม|โรค)xP(โรค)

    ## 1) หา P(มุม|โรค)
    view = ['FP-A', 'FP-B', 'FP-C', 'FP-D', 'FP-E']
    Class15AB = ['AB01', 'AB02', 'AB03', 'AB04', 'AB05', 'AB06', 'AB07', 'AB081', 'AB082', 'AB083', 'AB09', 'AB10', 'AB11', 'AB12', 'Normal']

        # Laplacian correction Adding 1 to each case
    p_fp_ = []
    fp_ = []
    AB_ = []
    for i in range(15) :
        fp = [1,1,1,1,1]
        FP = list(conf_mat.iloc[:,i][0:5])
        FP_ = [a +b for a, b in zip(FP, fp)]
        C_all =  sum(FP_)
        view = ['FP-A', 'FP-B', 'FP-C', 'FP-D', 'FP-E']
        for j in range(len(FP_)):
            a = FP_[j]
            p_fp_.append(a)
            AB_.append(Class15AB[i])
            fp_.append(view[j])

    prob_Subclass_Views = np.asarray(p_fp_).reshape((15, 5))
    bb = prob_Subclass_Views.T
    df = pd.DataFrame(bb, columns = ['AB01', 'AB02', 'AB03', 'AB04', 'AB05', 'AB06', 'AB07', 'AB081', 'AB082', 'AB083', 'AB09', 'AB10', 'AB11', 'AB12', 'Normal'])

        # หา prob. มุมในแต่ละโรค

    p_fp_ = []
    fp_ = []
    AB_ = []
    for i in range(15) :
        fp = [1,1,1,1,1]
        FP = list(conf_mat.iloc[:,i][0:5])
        FP_ = [a +b for a, b in zip(FP, fp)]
        C_all =  sum(FP_)
        view = ['FP-A', 'FP-B', 'FP-C', 'FP-D', 'FP-E']
        for j in range(len(FP_)):
            a = FP_[j]/C_all
            p_fp_.append(a)
            AB_.append(Class15AB[i])
            fp_.append(view[j])

    prob_Subclass_Views = np.asarray(p_fp_).reshape((15, 5))
    bb = prob_Subclass_Views.T
    df = pd.DataFrame(bb, columns = ['AB01', 'AB02', 'AB03', 'AB04', 'AB05', 'AB06', 'AB07', 'AB081', 'AB082', 'AB083', 'AB09', 'AB10', 'AB11', 'AB12', 'Normal'])

    ## 2) หา P(โรค)

    count_ = df_[['Sub_class_New','Unnamed: 0']].groupby(['Sub_class_New']).count().reset_index()

    C = count_['Unnamed: 0'].tolist()
    sum_C = count_['Unnamed: 0'].sum()
    prob = []
    for i in C:
        prob.append(i/sum_C)
    count_['prob_15AB'] = prob

    # 3) P(โรค|มุม)

    P_15AB = count_
    prob_15AB = P_15AB['prob_15AB'].tolist()
    prob_15AB = np.asarray(prob_15AB).reshape((15, 1))

    prob_15AB_FP = prob_15AB * prob_Subclass_Views

        ## transpose matrix
    b = prob_15AB_FP.T
    df = pd.DataFrame(b, columns = ['AB01', 'AB02', 'AB03', 'AB04', 'AB05', 'AB06', 'AB07', 'AB081', 'AB082', 'AB083', 'AB09', 'AB10', 'AB11', 'AB12', 'Normal'])

        ## Set normal value = 1
    FP_ = []
    for j in range(5) :
        FP = list(df.iloc[j,:][0:14])
        fp = []
        if j == 4:
            fp = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,1,1]
            FP_.append(fp)
        else:
            for i in FP:
                fp.append(i)
            fp.append(1)
            FP_.append(fp)

    FP = np.array(FP_)
    dfs_ = pd.DataFrame(FP, columns = ['AB01', 'AB02', 'AB03', 'AB04', 'AB05', 'AB06', 'AB07', 'AB081', 'AB082', 'AB083', 'AB09', 'AB10', 'AB11', 'AB12', 'Normal'])
#     FP = np.array(dfs_)
    
#         ## matrix softmax
#     FP_ = []
#     for j in range(5) :
#         FP = list(dfs_.iloc[j,:][0:14])
#         fp = []
#         if j == 4:
#             fp = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,1,1]
#             FP_.append(fp)
#         else:
#             a = softmax(FP)
#             for i in a:
#                 fp.append(i)
#             fp.append(1)
#             FP_.append(fp)
#     FP = np.array(FP_)
    
    ## หาร SUM ยกเว้น normal 
    FP_ = []
    for j in range(5) :
        FP = list(dfs_.iloc[j,:][0:14])
        fp = []
        fp_max = sum(FP)
        if j == 4:
            fp = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,1,1]
            FP_.append(fp)
        else:
            for i in FP:
                fp.append(i/fp_max)
            fp.append(1)
            FP_.append(fp)
    FP = np.array(FP_)
    
    return FP

In [8]:
TB = bayes_softmax(dataframe)
TB

array([[0.18111135, 0.23400243, 0.08821787, 0.17866663, 0.07215773,
        0.12058911, 0.00608908, 0.04790554, 0.04557958, 0.01747011,
        0.00206165, 0.00194112, 0.0021189 , 0.00208889, 1.        ],
       [0.24146064, 0.18770515, 0.04791817, 0.06909656, 0.108791  ,
        0.01043173, 0.09130241, 0.10450432, 0.08781971, 0.04030072,
        0.00535038, 0.0016792 , 0.00183298, 0.00180703, 1.        ],
       [0.15752389, 0.11507601, 0.04069271, 0.0739086 , 0.00929374,
        0.00181202, 0.04025864, 0.08638168, 0.07658387, 0.0227511 ,
        0.18029862, 0.07700372, 0.11653208, 0.00188331, 1.        ],
       [0.00324717, 0.00324194, 0.00312655, 0.01281334, 0.0031419 ,
        0.00306291, 0.0030932 , 0.00317421, 0.00315737, 0.00295822,
        0.0031419 , 0.00295822, 0.52312154, 0.42976151, 1.        ],
       [1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 

In [9]:
df_ = pd.DataFrame(TB, columns = [ 'AB01','AB02','AB03', 'AB04', 'AB05', 'AB06', 'AB07', 'AB081', 'AB082', 'AB083', 'AB09', 'AB10', 'AB11', 'AB12', 'Normal'])
df_

,AB01,AB02,AB03,AB04,AB05,AB06,AB07,AB081,AB082,AB083,AB09,AB10,AB11,AB12,Normal
0,0.181111,0.234002,0.088218,0.178667,0.072158,0.120589,0.006089,0.047906,0.045580,0.017470,0.002062,0.001941,0.002119,0.002089,1.0
1,0.241461,0.187705,0.047918,0.069097,0.108791,0.010432,0.091302,0.104504,0.087820,0.040301,0.005350,0.001679,0.001833,0.001807,1.0
2,0.157524,0.115076,0.040693,0.073909,0.009294,0.001812,0.040259,0.086382,0.076584,0.022751,0.180299,0.077004,0.116532,0.001883,1.0
3,0.003247,0.003242,0.003127,0.012813,0.003142,0.003063,0.003093,0.003174,0.003157,0.002958,0.003142,0.002958,0.523122,0.429762,1.0
4,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0
